# Ch 13-2 고객 세분화를 통한 마케팅 전략 수립

## 문제 상황

주문 내역 데이터를 바탕으로 고객들을 주문 특성과 주문 상품에 따라 그룹화하는 것이 목적

1. 주문 특성 기준 군집화
    1. 환불 데이터와 주문 데이터로 분할
    2. 특징 부착
    3. 코사인 유사도 기반의 계층 군집화 수행
2. 주문 상품 기준 군집화
    1. 판매 상위 100개 상품 출형 여부를 나타내는 데이터로 변환
    2. 자카드 유사도 기반의 계층 군집화 수행

## 1. 주문 특성 기준 군집화

### 데이터 분할

- 해당 데이터에는 주문 데이터와 환불 데이터가 동시에 포함되어 있기 때문에
- 환불 데이터는 주문 수량이 음수라는 특징이 있어, 데이터를 주문과 환불로 분할하는데 활용


### 특징 추출

1. 군집화 데이터를 유니크한 고객 ID 컬럼만 있는 데이터프레임으로 초기화
2. 주문/반품 횟수 계산 및 부착: 고객 ID와 주문 ID를 기준으로 중복을 제거하는 방식으로 유니크한 고객 ID와 주문 ID를 추출한 뒤, 추출한 데이터에서 고객 ID의 수를 카운트하여 군집화 데이터에 부착
3. 주문량 계산 및 부착: 고객 ID에 따른 주문량의 합계를 계산하여 군집화 데이터에 부착
4. 주문 금액 계산 및 부착: 주문량과 단가를 곱하여 주문 금액을 계산한 뒤, 고객 ID에 따른 주문 금액의 합계를 계산하여 군집화 데이터에 부착
5. 최근성 계산 및 부착: 현재 날짜에서 주문 날짜의 차이를 뺀 뒤, 고객별 해당 값의 최소 값(얼마나 최근에 샀는지)을 군집화 데이터에 부착


### 코사인 유사도 기반의 계층 군집화 수행

군집의 파라미터를 설정하고, 군집별 주요 특성을 확인한다. 

군집화 데이터에 (군집 개수 = 5, 군집 간 거리 = 평균, 거리척도 = 코사인 유사도)를 갖는 군집화 모델을 학습하여, 군집별 주요 특성을 파악한다.

## 2. 주문 상품 기준 군집화

주문 횟수가 상위 100등 안에 드는 상품들을 기준으로 고객을 상품 구매 여부로 구성된 벡터로 표현할 것. 

한 고객이 상품을 샀고 안샀고 하는 형태로 정리한 후, 

군집화 데이터에 (군집 개수 = 5, 군집 간 거리 = 평균, 거리척도 = 자카드 유사도)를 갖는 군집화 모델을 학습하여, 군집별 주요 특성을 파악한다.

대부분 0을 갖는 희소한 데이터이기 때문에 자카드 유사도를 사용한다.

In [1]:
import pandas as pd
import numpy as np
from scipy.stats import *

import os
os.chdir(r"/Users/Angela/Desktop/과속대학쿠쿠루/2. 탐색적 데이터 분석/데이터")

print(pd.__version__)
print(np.__version__)

1.4.1
1.22.4


In [2]:
df = pd.read_csv("E-Commerce_UK.csv")
df.head()

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,12/1/2010 8:26,2.55,C-17850
1,536365,71053,WHITE METAL LANTERN,6,12/1/2010 8:26,3.39,C-17850
2,536365,84406B,CREAM CUPID HEARTS COAT HANGER,8,12/1/2010 8:26,2.75,C-17850
3,536365,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,6,12/1/2010 8:26,3.39,C-17850
4,536365,84029E,RED WOOLLY HOTTIE WHITE HEART.,6,12/1/2010 8:26,3.39,C-17850


수량에 음수값이 들어간 경우는 그만큼 환불인 경우를 뜻한다. 

# 1. 주문 특성 기준 군집화

## 데이터 분할
주문 데이터와 환불 데이터로 분할한다.

In [3]:
refund_df = df.loc[df['Quantity']<=0]
order_df = df.loc[df['Quantity']>0]

## 고객별 특징 추출
클러스터링을 위한 데이터 초기화(라고 말하지만 새로 데이터 프레임을 만든다.)  
1. 고객ID 유니크값만으로 이루어진 데이터프레임을 만든다.  
### 주문 횟수
2. 주문데이터 `order_df`에서 고객ID와 송장넘버를 기준으로 중복을 제거한 후, 남은 고객아이디의 갯수를 센것이 아이디 별 주문 횟수가 되기 때문에 이 값을 `number_of_order_per_CID`로 정의했다.  
    이 `number_of_order_per_CID`는 인덱스가 고객ID, 밸류가 등장 횟수일 것이다. 이것은 시리즈 상태이기 때문에 replace에 그대로 들어갈 수 없기 때문에 to_dict를 사용한다. 그래서 이것을 위에서 만든 데이터프레임의 새 컬럼으로 주문횟수를 넣는다.  
    주문횟수가 0인 경우, 즉 환불만 있는 경우는 `number_of_order_per_CID`에 고객ID가 없을 것이다. 그것을 방지하기 위해 바뀌지 않은 것들, `Customer_ID`가 바뀌지 않은 것들은 전부 0으로 바꾸었다. 
    (주문 횟수가 0인 경우에는 replace가 되지 않아 CustomerID가 부착될 수 있음. 따라서 이러한 경우를 대비하기 위해 0으로 변경)
### 반품 횟수
3. 같은 방식으로 `refund_df`에서 반품횟수를 카운트한다.  
    마찬가지로 반품 횟수가 없다면 바뀌지 않았으므로 CustomerID가 부착되어있을 수 있기 때문에 이를 대비하여 0으로 바꾼다.  
### 총 주문량
4. 주문량은 `Quantity`의 합이다. 여러개의 상품을 구매한 것이므로 이것의 합을 `number_of_quantity_per_CID`으로 정의한다.  
    이 값은 시리즈 상태이기 때문에 마찬가지로 dictionary 형태로 변환후 붙이도록 한다. 
 마찬가지로 고객 ID로 붙은 경우는 전부 0으로 처리되도록 한다.  
### 총 주문 금액
5. `'주문 금액'`의 컬럼을 생성하고 주문 데이터`order_df`에서 각각 `Quantity`, `UnitPrice`가 있다. 이를 각각 곱한 방식으로 추가되도록 한다. 
    이때 추가한 것들을 고객ID를 기준으로 합하여서 고객별 총 주문금액을 구한다.  
    구한 것 시리즈를 변수로 저장해둔 후, 위와 같은 방식으로 새 데이터 프레임에 추가한다.  
### 가장 최근 주문
6. `to_datetime()`을 이용해 데이터 수집 날짜를 날짜 자료형으로 바꾸어 변수에 저장해둔다. 
7. 원 데이터의 invoicedate 컬럼에서 현재 날짜 변수를 빼도록 할 것인데, , `recency()`라는 함수를 만든다. apply를 사용하기 위한 함수이다.
    invoicedate컬럼의 날짜가 월 일 년 시간으로 이루어져 있다. 이를 split으로 자르고,  
    자른 데이터를 년-월-일 순서로 들어가도록 포맷을 지정해주었다. 
    그리고 현재 날짜 변수에서 이를 빼고, 뺸 것을 `days`(datetime에 있는 attribute)를 이용하여 '일'만 계산하도록 한다.  
    계산 결과 날짜 차이를 반환한다.  
8. 송장날짜를 현재와 날짜차이를 구하여 가장 작은 값을 가져오도록 하는 방법이 있지만,  
9. 우리는 날짜만 필요하기 때문에 중복을 최소화하여 계산량을 줄이려 한다.  
    `CID`, `InvoiceDate` 기준으로 중복을 제거하는데, 가장 나중의 것만 유지하도록 `keep = 'last'`를 사용하여 `CID`, `InvoiceDate`만 가져온다.  
    그리고 이를 `recency()` 함수를 적용하여 결과값을 `'최근성'`이라는 컬럼에 저장하도록 한다.  
10. 그리고 최근성을 밸류로, CID를 인덱스로 갖는 시리즈를 만들고, 이를 딕셔너리로 변환하여 위와 같은 방식으로 `replace`를 이용하여 덮는다.

In [5]:
# 클러스터링을 위한 데이터 초기화
cluster_df = pd.DataFrame({"CustomerID":order_df['CustomerID'].unique()})
cluster_df.head()

,CustomerID
0,C-17850
1,C-13047
2,C-12583
3,C-13748
4,C-15100


In [6]:
# 주문 횟수 계산 및 부착
number_of_order_per_CID = order_df.drop_duplicates(subset = ['CustomerID', 'InvoiceNo'])['CustomerID'].value_counts()
cluster_df['주문횟수'] = cluster_df['CustomerID'].replace(number_of_order_per_CID.to_dict())

# 주문 횟수가 0인 경우에는 replace가 되지 않아 CustomerID가 부착될 수 있음
# 따라서 이러한 경우를 대비하기 위해 0으로 변경
cluster_df.loc[cluster_df['CustomerID'] == cluster_df['주문횟수'], '주문횟수'] = 0
cluster_df.head()

,CustomerID,주문횟수
0,C-17850,34
1,C-13047,21
2,C-12583,15
3,C-13748,5
4,C-15100,4


In [7]:
# 반품 횟수 계산 및 부착 
number_of_refund_per_CID = refund_df.drop_duplicates(subset = ['CustomerID', 'InvoiceNo'])['CustomerID'].value_counts()
cluster_df['반품횟수'] = cluster_df['CustomerID'].replace(number_of_refund_per_CID.to_dict())

cluster_df.loc[cluster_df['CustomerID'] == cluster_df['반품횟수'], '반품횟수'] = 0
cluster_df.head()

,CustomerID,주문횟수,반품횟수
0,C-17850,34,1
1,C-13047,21,8
2,C-12583,15,3
3,C-13748,5,0
4,C-15100,4,3


In [8]:
# 주문량 계산 및 부착
number_of_quantity_per_CID = order_df.groupby('CustomerID')['Quantity'].sum()
cluster_df['주문량'] = cluster_df['CustomerID'].replace(number_of_quantity_per_CID.to_dict())
cluster_df.loc[cluster_df['CustomerID'] == cluster_df['주문량'], '주문량'] = 0
cluster_df.head()

,CustomerID,주문횟수,반품횟수,주문량
0,C-17850,34,1,1733
1,C-13047,21,8,1953
2,C-12583,15,3,5060
3,C-13748,5,0,439
4,C-15100,4,3,81


In [9]:
# 주문금액합계 계산 및 부착
order_df.loc[:, '주문금액'] = order_df.loc[:, 'Quantity'] * order_df.loc[:, 'UnitPrice']

number_of_quantity_per_CID = order_df.groupby('CustomerID')['주문금액'].sum()
cluster_df['주문금액합계'] = cluster_df['CustomerID'].replace(number_of_quantity_per_CID.to_dict())
cluster_df.loc[cluster_df['CustomerID'] == cluster_df['주문금액합계'], '주문금액합계'] = 0
cluster_df.head()

/var/folders/qw/j72d9nwn5dq0vxtqtw2bbtnh0000gp/T/ipykernel_6918/342078542.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  order_df.loc[:, '주문금액'] = order_df.loc[:, 'Quantity'] * order_df.loc[:, 'UnitPrice']


,CustomerID,주문횟수,반품횟수,주문량,주문금액합계
0,C-17850,34,1,1733,5391.21
1,C-13047,21,8,1953,6619.51
2,C-12583,15,3,5060,7281.38
3,C-13748,5,0,439,948.25
4,C-15100,4,3,81,877.25


In [10]:
# 최근 주문 ~ 현재 시점까지 거리 부착
current_date = pd.to_datetime('2011-12-10')
def recency(value):   # 하나의 element를 처리함!
    month, day, year = value.split(' ')[0].split('/')    
    diff = (current_date - pd.to_datetime('{}-{}-{}'.format(year, month, day))).days
    return diff

# keep = last로 계산량 감소
order_df_without_duplicates = order_df.drop_duplicates(subset = ['CustomerID', 'InvoiceDate'], keep = 'last')[['CustomerID', 'InvoiceDate']]
order_df_without_duplicates['최근성'] = order_df_without_duplicates['InvoiceDate'].apply(recency)

min_recency_per_CID = order_df_without_duplicates.set_index('CustomerID')['최근성']
cluster_df['최근성'] = cluster_df['CustomerID'].replace(min_recency_per_CID.to_dict())

CID는 군집화를 할 때 필요한 정보가 아니기 때문에 `set_index`를 사용하여 인덱스로 넣어버린다. 

In [11]:
cluster_df.head()

,CustomerID,주문횟수,반품횟수,주문량,주문금액합계,최근성
0,C-17850,34,1,1733,5391.21,373
1,C-13047,21,8,1953,6619.51,32
2,C-12583,15,3,5060,7281.38,3
3,C-13748,5,0,439,948.25,96
4,C-15100,4,3,81,877.25,331


In [12]:
cluster_df.set_index('CustomerID', inplace = True)

## 고객의 주문 특성에 따른 군집화 수행
군집화 모델 인스턴스화 및 학습


In [21]:
from sklearn.cluster import AgglomerativeClustering as AC
clu_model = AC(n_clusters = 5, 
                   affinity = 'cosine', 
                   linkage = 'average')
clu_model.fit(cluster_df)

AgglomerativeClustering(affinity='cosine', linkage='average', n_clusters=5)

In [22]:
cluster_df['주문특성_군집'] = clu_model.labels_ # 각 군집 정보를 df에 저장
cluster_df.head()

,주문횟수,반품횟수,주문량,주문금액합계,최근성,주문특성_군집
CustomerID,,,,,,
C-17850,34,1,1733,5391.21,373,3
C-13047,21,8,1953,6619.51,32,3
C-12583,15,3,5060,7281.38,3,3
C-13748,5,0,439,948.25,96,3
C-15100,4,3,81,877.25,331,2


### 해석
군집별로 어떤 특성이 있는지를 볼 것인데, 계층 군집화 모델은 중심이라는 개념이 없기 때문에 직접 구해야 한다. 그래서 평균을 구한다. 


In [23]:
cluster_df.groupby(['주문특성_군집'])['주문횟수', '반품횟수', '주문량', '주문금액합계', '최근성'].mean()

/var/folders/qw/j72d9nwn5dq0vxtqtw2bbtnh0000gp/T/ipykernel_6918/2420321682.py:1: FutureWarning: Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.
  cluster_df.groupby(['주문특성_군집'])['주문횟수', '반품횟수', '주문량', '주문금액합계', '최근성'].mean()


,주문횟수,주문량,주문금액합계,최근성
주문특성_군집,,,,
0,1.151111,72.773333,162.506978,267.486667
1,1.904762,1486.809524,375.164762,102.476190
2,1.513937,175.416376,334.699895,190.958188
3,5.773610,1606.724251,3278.710038,52.331359
4,3.796154,2080.311538,1652.155385,50.138462


# 주문 상품 기준 군집화 
### 주요 상품 확인